In [1]:
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import re

import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer

from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras import layers
from tensorflow.python.keras.optimizers import adam_v2

In [ ]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip glove*

--2022-11-07 17:50:24--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2022-11-07 17:50:24--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2022-11-07 17:50:25--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/

In [ ]:
# Loading dataframes
df_train = pd.read_csv('train.csv', names=['text', 'label'])
df_dev = pd.read_csv('dev.csv', names=['text', 'label'])
df_test = pd.read_csv('test.csv', names=['text', 'label'])

df_train.dropna()
df_dev.dropna()
df_test.dropna()

# Loading dataframes into variables
x_train, y_train, x_dev, y_dev, x_test, y_test = df_train['text'], df_train['label'], df_dev['text'], df_dev['label'], df_test['text'], df_test['label']

In [ ]:
encoder = LabelBinarizer()

Y_train_bin = encoder.fit_transform(y_train)  
Y_dev_bin = encoder.fit_transform(y_dev)
Y_test_bin = encoder.fit_transform(y_test)

In [ ]:
max_comment_length = 200

tokenizer = tf.keras.preprocessing.text.Tokenizer()

tokenizer.fit_on_texts(df_train['text'])
tokenizer.fit_on_texts(df_dev['text'])
tokenizer.fit_on_texts(df_test['text'])

x_train_sequences = tokenizer.texts_to_sequences(df_train['text'])
x_validation_sequences = tokenizer.texts_to_sequences(df_dev['text'])
x_test_sequences = tokenizer.texts_to_sequences(df_test['text'])

x_train_padded = tf.keras.preprocessing.sequence.pad_sequences(x_train_sequences, maxlen=max_comment_length)
x_validation_padded = tf.keras.preprocessing.sequence.pad_sequences(x_validation_sequences, maxlen=max_comment_length)
x_test_padded = tf.keras.preprocessing.sequence.pad_sequences(x_test_sequences, maxlen=max_comment_length)


In [ ]:
def get_glove_embedding_vectors(glove_filename):
  embedding_vectors = {}

  with open(glove_filename,'r',encoding='utf-8') as file:
      for row in file:
          values = row.split(' ')
          word = values[0]
          weights = np.asarray([float(value) for value in values[1:]])
          embedding_vectors[word] = weights
  return embedding_vectors

In [ ]:
embedding_vectors = get_glove_embedding_vectors('glove.840B.300d.txt')

In [ ]:
embedding_dim = 300
vocab_length = len(tokenizer.word_index)+1

embedding_matrix = np.zeros((vocab_length, embedding_dim))
for word, index in tokenizer.word_index.items():
    if index < vocab_length:
        embedding_vector = embedding_vectors.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
opt = adam_v2.Adam(learning_rate=0.001)

def create_model():
    model = Sequential()
    model.add(Embedding(vocab_length, embedding_dim, trainable = False, weights=[embedding_matrix]))
    model.add(LSTM(512))
    model.add(Dropout(0.3))
    model.add(layers.Dense(2, activation='softmax' ))
    model.compile(loss='sparse_categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

    return model

model = create_model()
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 300)         6317100   
_________________________________________________________________
lstm_11 (LSTM)               (None, 512)               1665024   
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 1026      
Total params: 7,983,150
Trainable params: 1,666,050
Non-trainable params: 6,317,100
_________________________________________________________________


In [ ]:
from tensorflow.python.keras.callbacks import EarlyStopping 

early_stopping = EarlyStopping()
history = model.fit(pd.DataFrame(x_train_padded), Y_train_bin,
                    epochs=2,
                    verbose=True,
                    shuffle=True,
                    validation_data=(pd.DataFrame(x_validation_padded), Y_dev_bin),
                    batch_size=100,
                    #callbacks = [early_stopping]
)

Epoch 1/2
123/123 [==============================] - 105s 840ms/step - loss: 0.4613 - accuracy: 0.7837 - val_loss: 0.4638 - val_accuracy: 0.7860
Epoch 2/2
123/123 [==============================] - 101s 823ms/step - loss: 0.4363 - accuracy: 0.7993 - val_loss: 0.4771 - val_accuracy: 0.7740


In [ ]:
metrics = model.evaluate(pd.DataFrame(x_test_padded), Y_test_bin)

27/27 [==============================] - 2s 80ms/step - loss: 0.4307 - accuracy: 0.7988


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
predictions = model.predict_classes(pd.DataFrame(x_test_padded))

test_labels_transposed = []
for x in Y_test_bin:
  test_labels_transposed.append(int(x))

report = classification_report(test_labels_transposed , predictions,output_dict=True, digits=4)
report = pd.DataFrame(report).transpose()
print(report.to_latex())

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:454: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


\begin{tabular}{lrrrr}
\toprule
{} &  precision &    recall &  f1-score &     support \\
\midrule
0            &   0.866995 &  0.851613 &  0.859235 &  620.000000 \\
1            &   0.633466 &  0.662500 &  0.647658 &  240.000000 \\
accuracy     &   0.798837 &  0.798837 &  0.798837 &    0.798837 \\
macro avg    &   0.750231 &  0.757056 &  0.753446 &  860.000000 \\
weighted avg &   0.801824 &  0.798837 &  0.800190 &  860.000000 \\
\bottomrule
\end{tabular}

